# Use fitsio instead of PyFITS to read outputs from CoLoRe

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
import fitsio
import matplotlib.pyplot as plt

In [3]:
#Open data file (test file with few quasars)
h = fitsio.FITS('../example_data/raw_colore/test_N1000.fits')

In [4]:
# get information about quasars (TYPE,RA,DEC,Z_COSMO,DZ_RSD)
catalog = h[1].read()
z_qso = catalog['Z_COSMO']
Nq = len(z_qso)
print('# quasars =',Nq)
print(np.min(z_qso),'< z_qso <',np.max(z_qso))

# quasars = 1000
0.19643 < z_qso < 3.74638


In [5]:
# get arraw with redshift in each cell of grid
z = h[4].read()['Z']
Nz=len(z)

In [6]:
# Get deltas (fluctuation around mean density) and line of sight velocity (actually, dz)
deltas = h[2].read()
velocities = h[3].read()

In [7]:
flux = np.exp(-1-deltas)

In [8]:
filename='desisim_lya_N10.fits'

In [9]:
fits = fitsio.FITS(filename,'rw')

In [10]:
# write skewers for desisim
N=10
for i in range(N):
    data = {}
    data['LAMBDA']=1215.67*(1+z)
    data['FLUX']=flux[i]
    head = {}
    head['ZQSO']=z_qso[i]
    head['RA']=catalog['RA'][i]
    head['DEC']=catalog['DEC'][i]
    head['MAG_G']=21
    fits.write(data,header=head)
fits.close()

In [11]:
# test whether we can read file as in desisim.lya_spectra
test = fitsio.FITS(filename)
heads = []
for i in range(N):
    heads.append(test[i+1].read_header())    
fits.close()

zqso = np.array([head['ZQSO'] for head in heads])
ra = np.array([head['RA'] for head in heads])
dec = np.array([head['DEC'] for head in heads])
mag_g = np.array([head['MAG_G'] for head in heads])

for i in range(N):
    data = test[i + 1].read()
    la = data['LAMBDA'][:]
    tr = data['FLUX'][:]